# Exercice 1.5

Your next goal is to run the following query to get the population
information of different countries (limited to 10000 rows). Run the
following query on [Wikidata query service](https://query.wikidata.org)
and download the JSON file.

```SPARQL
SELECT DISTINCT ?countryLabel (YEAR(?date) as ?year) ?population
WHERE {
 ?country wdt:P31 wd:Q6256; #Country 
   p:P1082 ?populationStatement;
  rdfs:label ?countryLabel. #Label
 ?populationStatement ps:P1082 ?population; #population
  pq:P585 ?date. #period in time
 FILTER(lang(?countryLabel)="en") #Label in English
}
ORDER by ?countryLabel ?year
LIMIT 10000
```

In [1]:
from pandas import json_normalize
import pandas as pd
import json

In [2]:
# Ouvrture du fichier
data = json.load(open("query_1.json"))
dataframe = json_normalize(data)
dataframe

,countryLabel,year,population
0,Afghanistan,1960,8774440
1,Afghanistan,1961,8953544
2,Afghanistan,1962,9141783
3,Afghanistan,1963,9339507
4,Afghanistan,1964,9547131
...,...,...,...
8584,Zimbabwe,2010,13076978
8585,Zimbabwe,2011,13358738
8586,Zimbabwe,2012,13724317
8587,Zimbabwe,2013,14149648


## The population of countries in alphabetical order of their names and ascending order of year.

In [3]:
dataframe.sort_values(["countryLabel", "year"], ascending = (True, True))

,countryLabel,year,population
0,Afghanistan,1960,8774440
1,Afghanistan,1961,8953544
2,Afghanistan,1962,9141783
3,Afghanistan,1963,9339507
4,Afghanistan,1964,9547131
...,...,...,...
8584,Zimbabwe,2010,13076978
8585,Zimbabwe,2011,13358738
8586,Zimbabwe,2012,13724317
8587,Zimbabwe,2013,14149648


## The latest available population of every country

In [4]:
idx = dataframe.groupby(["countryLabel"], sort = False)["year"].transform(max) == dataframe["year"]
dataframe[idx]

,countryLabel,year,population
57,Afghanistan,2021,37466414
113,Albania,2017,2873457
170,Algeria,2020,43900000
226,American Samoa,2020,55197
285,Andorra,2021,78151
...,...,...,...
8441,Vietnam,2018,94660000
8469,Wales,2016,3113000
8476,Yemen,2017,28250420
8533,Zambia,2017,17094130


## The country with the lowest and highest population (considering the latest population)

In [5]:
# Parsing "population" column to integer
dataframe["population"] = dataframe["population"].astype(int)
# We take the latest available population record for every country
ndf = dataframe[dataframe.groupby(["countryLabel"], sort = False)["year"].transform(max) == dataframe["year"]]
# We sort values by population
sorted_dataframe = ndf.sort_values(by=["population"])
first_and_last_elements = sorted_dataframe.iloc[[0, -1]]
print(first_and_last_elements)

                    countryLabel  year  population
5605                        Niue  2016        1612
6075  People's Republic of China  2021  1442965000


# Exercice 1.5.2
```SPARQL
SELECT ?title ?subjectLabel ?year
{
  ?article wdt:P31 wd:Q13442814; #scientific article
           wdt:P1476 ?title; #title of the article
           wdt:P921 ?subject; #main subject
           wdt:P577 ?date. #publication date
  ?subject rdfs:label ?subjectLabel.
  BIND(YEAR(?date) as ?year).
  #published after 2010
  FILTER(lang(?title)="en" &&
     lang(?subjectLabel)="en" && ?year>2010)
}
LIMIT 10000
```

In [6]:
# Data loading
data = json.load(open("query_2.json"))
dataframe = json_normalize(data)
dataframe

,title,subjectLabel,year
0,Self-injurious thoughts and behaviors as risk ...,death,2015
1,Violent Deaths Among Georgia Workers: An Exami...,death,2016
2,Increasing Morbidity in the Pregnant Populatio...,death,2015
3,Pregnancy-Associated Deaths in Virginia Due to...,death,2016
4,Feeding the dead?,death,2015
...,...,...,...
4995,Regional variability in outpatient antibiotic ...,Canada,2018
4996,Residential moves and its association with sub...,Canada,2018
4997,Dermatology-Related Uses of Medical Cannabis P...,Canada,2018
4998,Increased precipitation drives mega slump deve...,Canada,2015


## The number of articles published on different subjects every year.

In [7]:
ndf = dataframe.groupby(["subjectLabel", "year"]).size().reset_index(name='counts')
ndf

,subjectLabel,year,counts
0,Africa,2011,6
1,Africa,2012,7
2,Africa,2013,15
3,Africa,2014,35
4,Africa,2015,25
5,Africa,2016,4
6,Africa,2017,12
7,Africa,2018,14
8,Africa,2019,22
9,Africa,2020,19


## Top subject of interest to the scientific community every year (based on the above query results).

In [12]:
df = ndf.groupby(["year", "subjectLabel"]).max("counts").sort_values("counts").groupby(["year"]).tail(1).sort_values("year", ascending=True).reset_index()
df

,year,subjectLabel,counts
0,2011,Canada,481
1,2012,Canada,537
2,2013,Canada,598
3,2014,Canada,538
4,2015,Canada,546
5,2016,Canada,556
6,2017,Canada,260
7,2018,happiness,146
8,2019,happiness,135
9,2020,happiness,39


## Top 10 subjects of interest to the scientific community (based on the above query results) since 2010.

In [25]:
# filtre les années < 2010 puis on prend les 10 premiers éléments les plus grand
df.loc[df["year"] >= "2010"].sort_values(by="counts").tail(10).reset_index()


,index,year,subjectLabel,counts
0,9,2020,happiness,39
1,8,2019,happiness,135
2,7,2018,happiness,146
3,6,2017,Canada,260
4,0,2011,Canada,481
5,1,2012,Canada,537
6,3,2014,Canada,538
7,4,2015,Canada,546
8,5,2016,Canada,556
9,2,2013,Canada,598
